In [127]:
#!/usr/bin/env python3

import numpy as np
import copy
import matscipy.calculators.eam.calculator
import shelve
import os

#def find_peak():
 #   os.system('gen-ns')
  #  os.system('./ns_analyse 64Cu.energies -M 0 -n 600 -D 5 > pf')
  #  T,Cp = numpy.loadtxt('pf', usecols=(0,3), unpack=True)
   # index_PT =  numpy.where(Cp == numpy.amax(Cp))
    #print(int(T[index_PT]))
    #return(experimental - int(T[index_PT]))

def gen_poten(param):
    potential = matscipy.calculators.eam.io.read_eam('Cu_Zhou04.eam.alloy', 'eam/alloy')
    matscipy.calculators.eam.io.write_eam(potential[0], potential[1], (potential[2] * param[0]), (potential[3] * param[1]), (potential[4] *  param[2]), 'test.eam.alloy','eam/alloy')

def save_variables():
    cwd = os.getcwd
    filename= str(cwd) + '/variables'
    shelf = shelve.open(filename, 'n')
    for key in globals():
        try:
            shelf[key] = globals()[key]
        except:
            print('ERROR shleving: {0}'.format(key))
    shelf.close()

def load_variables():
    cwd = os.getcwd()
    shelf = shelf.open(str(cwd +'/variables'))
    for key in shelf:
        globals()[key] = shelf[key]
    shelf.close


In [128]:
def find_peak(x):
    """the function of which integrand we want to find this is a 2D function, with three gaussians"""
    f=np.exp(-0.2*(x[0]-3)**2-0.2*(x[1]-5)**2) + 0.9*np.exp(-0.5*(x[0]-7)**2-0.5*(x[1]-7)**2) + 0.6*np.exp(-(x[0]-5)**2-(x[1]-3)**2)
    f = abs(1 - f)
    return f

In [129]:
step = 0.5
no_improv_thr = 10e-6
no_improv_break= 2
max_iter = 1000
alpha = 1.
gamma = 2.
rho = -0.5
sigma = 0.5

x_start = np.array([1.0, 1.0])
print(x_start)
print(x_start.shape)
dim = len(x_start)
prev_best = find_peak(x_start)
no_improv = 0
res = [[x_start, prev_best]]

[1. 1.]
(2,)


In [130]:
print(x_start)
print(type(x_start))

[1. 1.]
<class 'numpy.ndarray'>


In [131]:
for i in range(dim):
    x = copy.copy(x_start)
    x[i] = x[i] + step
    score = find_peak(x)
    res.append([x, score])
    print(score)
    print('\n', str(res))
    

0.9740088186357516

 [[array([1., 1.]), 0.9816843598745735], [array([1.5, 1. ]), 0.9740088186357516]]
0.9612257850516022

 [[array([1., 1.]), 0.9816843598745735], [array([1.5, 1. ]), 0.9740088186357516], [array([1. , 1.5]), 0.9612257850516022]]


In [132]:
# simplex iter
iters = 0
while 1:
    print('Best parameters so far:', str(res[-1][0]))
    print('iteration:', str(iters))
    # order
    res.sort(key=lambda x: x[1])
    best = res[0][1]
    print(max_iter)
    # break after max_iter
    if iters > max_iter:
        print(res[0])
        print('% off target: ', (best/1) *100)
        break
        exit()
    iters += 1

    # break after no_improv_break iterations with no improvement
    print( '...Distance from target:', str(best))

    if best < (prev_best - no_improv_thr):
        no_improv = 0
        prev_best = best
    else:
        no_improv += 1

    if no_improv >= no_improv_break:
        print('FINISHED')
        print('Parameters and score:', res[0])
        print('% off target: ', (best/1) *100)
        print('score: ', str(best +1))
        break
        exit()

    # centroid
    x0 = [0.] * dim
    for tup in res[:-1]:
        for i, c in enumerate(tup[0]):
            x0[i] += c / (len(res)-1)

    # reflection
    xr = x0 + alpha*(x0 - res[-1][0])
    rscore = find_peak(xr)
    if res[0][1] <= rscore < res[-2][1]:
        del res[-1]
        res.append([xr, rscore])
        continue

    # expansion
    if rscore < res[0][1]:
        xe = x0 + gamma*(x0 - res[-1][0])
        escore = find_peak(xe)
        if escore < rscore:
            del res[-1]
            res.append([xe, escore])
            continue
        else:
            del res[-1]
            res.append([xr, rscore])
            continue

    # contraction
    xc = x0 + rho*(x0 - res[-1][0])
    cscore = find_peak(xc)
    if cscore < res[-1][1]:
        del res[-1]
        res.append([xc, cscore])
        continue

    # reduction
    x1 = res[0][0]
    nres = []

    #for tup in res:
    i=0
    while i < len(res): # might be a bug here from the loop type change, check
        redx = x1 + sigma*(tup[0] - x1)
        score = find_peak(redx)
        nres.append([redx, score])
        i += 1
    res = nres


Best parameters so far: [1.  1.5]
iteration: 0
1000
...Distance from target: 0.9612257850516022
Best parameters so far: [1.75 1.75]
iteration: 1
1000
...Distance from target: 0.9115186276079266
Best parameters so far: [1.125 2.875]
iteration: 2
1000
...Distance from target: 0.7993612222535843
Best parameters so far: [2.3125 3.9375]
iteration: 3
1000
...Distance from target: 0.27389586984303627
Best parameters so far: [1.6875 5.0625]
iteration: 4
1000
...Distance from target: 0.27389586984303627
Best parameters so far: [2.875 6.125]
iteration: 5
1000
...Distance from target: 0.22592725968095828
Best parameters so far: [3.5 5. ]
iteration: 6
1000
...Distance from target: 0.047345862760498236
Best parameters so far: [2.75 4.75]
iteration: 7
1000
...Distance from target: 0.024503896425604577
Best parameters so far: [3.  5.5]
iteration: 8
1000
...Distance from target: 0.024503896425604577
Best parameters so far: [3.0625 5.1875]
iteration: 9
1000
...Distance from target: 0.007589797636970874